# API REST - SISTEMA DE DETECCIÓN DE FRAUDE TELEFÓNICO
## API completa con FastAPI para scoring, aprendizaje incremental y gestión del modelo

In [1]:
from fastapi import FastAPI, File, UploadFile, HTTPException, BackgroundTasks, Query
from fastapi.responses import JSONResponse, FileResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any
from datetime import datetime, timedelta
from enum import Enum
import pandas as pd
import numpy as np
import pickle
import os
import json
import asyncio
import logging
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Librerías importadas para API")

✅ Librerías importadas para API


In [2]:
import oracledb
from datetime import datetime
import os
from typing import Optional
from contextlib import contextmanager
import json

# ## CONFIGURACIÓN DE ORACLE DATABASE CON ORACLEDB


In [3]:
# Configuración de Oracle (ajusta según tu ambiente)
ORACLE_CONFIG = {
    "user": "SYSTEM",
    "password": "erick710132",
    "dsn": "localhost:1521/xe",  # Formato: host:port/service_name
    # Alternativa con TNS:
    # "dsn": "XEPDB1"  # Si usas tnsnames.ora
}

# Pool de conexiones para mejor rendimiento
oracle_pool = None

def inicializar_oracle_pool():
    """Inicializa el pool de conexiones de Oracle"""
    global oracle_pool
    try:
        # Crear pool de conexiones
        oracle_pool = oracledb.create_pool(
            user=ORACLE_CONFIG["user"],
            password=ORACLE_CONFIG["password"],
            dsn=ORACLE_CONFIG["dsn"],
            min=2,
            max=10,
            increment=1
        )
        logger.info("Pool de conexiones Oracle inicializado")
        return True
    except Exception as e:
        logger.error(f"Error inicializando Oracle: {e}")
        return False

@contextmanager
def get_oracle_connection():
    """Context manager para obtener conexión del pool"""
    connection = oracle_pool.acquire()
    connection.encoding = "UTF-8"
    connection.nencoding = "UTF-8"
    try:
        yield connection
    finally:
        oracle_pool.release(connection)

# ## FUNCIONES ACTUALIZADAS PARA ORACLEDB

In [4]:
def crear_tablas_oracle():
    """Crea las tablas necesarias en Oracle si no existen"""
    
    # Script SQL para crear las tablas
    sql_scripts = [

        """
        CREATE TABLE ALERTAS_FRAUDE (
            ID_ALERTA NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
            FECHA_PROCESAMIENTO TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FECHA_REGISTRO VARCHAR2(10),
            CODIGO_PAIS NUMBER,
            LINEA VARCHAR2(20),
            N_LLAMADAS NUMBER,
            N_MINUTOS NUMBER(10,2),
            N_DESTINOS NUMBER,
            SCORE_ANOMALIA NUMBER(10,4),
            UMBRAL NUMBER(10,4),
            TIPO_ANOMALIA VARCHAR2(50),
            TIPO_CONTEXTO VARCHAR2(50),
            RAZON_DECISION VARCHAR2(200),
            ARCHIVO_ORIGEN VARCHAR2(100),
            LOTE_PROCESAMIENTO VARCHAR2(50)
        )
        """,
        
        """
        CREATE TABLE LOTES_PROCESAMIENTO (
            ID_LOTE NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
            TIMESTAMP_INICIO TIMESTAMP,
            TIMESTAMP_FIN TIMESTAMP,
            ARCHIVO_ENTRADA VARCHAR2(200),
            TOTAL_REGISTROS NUMBER,
            TOTAL_ANOMALIAS NUMBER,
            TASA_ANOMALIAS NUMBER(5,2),
            ARCHIVO_SALIDA VARCHAR2(200),
            ESTADO VARCHAR2(20),
            MENSAJE_ERROR VARCHAR2(500)
        )
        """,
        
        """
        CREATE TABLE HISTORICO_UMBRALES (
            ID_CAMBIO NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
            FECHA_CAMBIO TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UMBRAL_ANTERIOR NUMBER(10,4),
            UMBRAL_NUEVO NUMBER(10,4),
            RAZON_CAMBIO VARCHAR2(200),
            USUARIO VARCHAR2(50)
        )
        """,
        
        """
        CREATE TABLE APRENDIZAJE_INCREMENTAL (
            ID_APRENDIZAJE NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
            FECHA_EJECUCION TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FECHA_DATOS DATE,
            ARCHIVO_ENTRADA VARCHAR2(200),
            
            -- Estadísticas del procesamiento
            TOTAL_REGISTROS NUMBER,
            REGISTROS_PROCESADOS NUMBER,
            PAISES_TOTALES NUMBER,
            PAISES_NUEVOS NUMBER,
            PAISES_ACTUALIZADOS NUMBER,
            
            -- Información del modelo antes
            UMBRAL_ANTERIOR NUMBER(10,4),
            N_TREES_ANTERIOR NUMBER,
            PAISES_CONOCIDOS_ANTERIOR NUMBER,
            
            -- Información del modelo después
            UMBRAL_NUEVO NUMBER(10,4),
            N_TREES_NUEVO NUMBER,
            PAISES_CONOCIDOS_NUEVO NUMBER,
            
            -- Cambios en estadísticas
            PAISES_CON_CAMBIO_CATEGORIA NUMBER,
            DETALLE_CAMBIOS_CATEGORIA CLOB,
            
            -- Métricas de performance
            TIEMPO_PROCESAMIENTO_SEG NUMBER(10,2),
            MEMORIA_USADA_MB NUMBER(10,2),
            
            -- Estado y respaldos
            ESTADO VARCHAR2(20),
            MENSAJE_ERROR VARCHAR2(500),
            ARCHIVO_BACKUP_CONFIG VARCHAR2(200),
            
            -- Estadísticas agregadas
            LLAMADAS_PROMEDIO_GLOBAL NUMBER(10,2),
            MINUTOS_PROMEDIO_GLOBAL NUMBER(10,2),
            DESTINOS_PROMEDIO_GLOBAL NUMBER(10,2),
            
            -- Umbrales de features
            UMBRAL_MINUTOS_EXTREMOS NUMBER(10,2),
            PESO_SPRAY_RATIO NUMBER(10,2),
            
            -- Observaciones
            OBSERVACIONES VARCHAR2(1000)
        )
        """,
        
        # Índices para mejorar performance
        "CREATE INDEX IDX_ALERTAS_FECHA ON ALERTAS_FRAUDE(FECHA_PROCESAMIENTO)",
        "CREATE INDEX IDX_ALERTAS_PAIS ON ALERTAS_FRAUDE(CODIGO_PAIS)",
        "CREATE INDEX IDX_ALERTAS_LINEA ON ALERTAS_FRAUDE(LINEA)",
        "CREATE INDEX IDX_ALERTAS_TIPO ON ALERTAS_FRAUDE(TIPO_ANOMALIA)",
        "CREATE INDEX IDX_APRENDIZAJE_FECHA ON APRENDIZAJE_INCREMENTAL(FECHA_EJECUCION)",
        "CREATE INDEX IDX_APRENDIZAJE_ESTADO ON APRENDIZAJE_INCREMENTAL(ESTADO)"
    ]
    
    with get_oracle_connection() as conn:
        cursor = conn.cursor()
        
        for script in sql_scripts:
            try:
                cursor.execute(script)
                logger.info(f"Tabla/Índice creado exitosamente")
            except oracledb.DatabaseError as e:
                error, = e.args
                if error.code == 955:  # Tabla ya existe
                    logger.info("Tabla ya existe, continuando...")
                else:
                    logger.error(f"Error creando tabla: {e}")
                    raise
        
        conn.commit()
        logger.info("Todas las tablas creadas/verificadas exitosamente")

# ## FUNCIONES DE GUARDADO EN ORACLE


In [5]:
def guardar_anomalias_oracle(anomalias_df, archivo_origen, lote_id):
    """Guarda las anomalías detectadas en Oracle"""
    
    if anomalias_df.empty:
        return
    
    with get_oracle_connection() as conn:
        cursor = conn.cursor()
        
        # Preparar datos para inserción
        datos_insertar = []
        for _, row in anomalias_df.iterrows():
            datos_insertar.append((
                row['FECHA'],
                int(row['CODIGODEPAIS']),
                row['LINEA'],
                int(row['N_LLAMADAS']),
                float(row['N_MINUTOS']),
                int(row['N_DESTINOS']),
                float(row['score_anomalia']),
                float(row['umbral']),
                row['tipo_anomalia'],
                row['tipo_contexto'],
                row['razon_decision'],
                archivo_origen,
                lote_id
            ))
        
        # Inserción masiva
        insert_sql = """
            INSERT INTO ALERTAS_FRAUDE (
                FECHA_REGISTRO, CODIGO_PAIS, LINEA, N_LLAMADAS, N_MINUTOS,
                N_DESTINOS, SCORE_ANOMALIA, UMBRAL, TIPO_ANOMALIA,
                TIPO_CONTEXTO, RAZON_DECISION, ARCHIVO_ORIGEN, LOTE_PROCESAMIENTO
            ) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13)
        """
        
        cursor.executemany(insert_sql, datos_insertar)
        conn.commit()
        
        logger.info(f"Se guardaron {len(datos_insertar)} anomalías en Oracle")

def registrar_lote_procesamiento(archivo_entrada, total_registros, total_anomalias, 
                               tasa_anomalias, archivo_salida, estado="COMPLETADO"):
    """Registra información del lote procesado"""
    
    with get_oracle_connection() as conn:
        cursor = conn.cursor()
        
        insert_sql = """
            INSERT INTO LOTES_PROCESAMIENTO (
                TIMESTAMP_INICIO, TIMESTAMP_FIN, ARCHIVO_ENTRADA, TOTAL_REGISTROS,
                TOTAL_ANOMALIAS, TASA_ANOMALIAS, ARCHIVO_SALIDA, ESTADO
            ) VALUES (
                :1, :2, :3, :4, :5, :6, :7, :8
            ) RETURNING ID_LOTE INTO :id_lote
        """
        
        id_lote = cursor.var(oracledb.NUMBER)
        cursor.execute(insert_sql, [
            datetime.now(),
            datetime.now(),
            archivo_entrada,
            total_registros,
            total_anomalias,
            tasa_anomalias,
            archivo_salida,
            estado,
            id_lote
        ])
        
        conn.commit()
        return id_lote.getvalue()[0]
    




# ## 1. CONFIGURACIÓN Y MODELOS DE DATOS

In [6]:
# Configuración de rutas
MODELS_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Modelos"
OUTPUT_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Resultados_API"
TEMP_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Temp_APII"
LOG_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Logs_API"

# Crear directorios si no existen
for path in [OUTPUT_PATH, TEMP_PATH, LOG_PATH]:
    os.makedirs(path, exist_ok=True)

# Configuración del modelo
modelo = None
scaler = None
config = None
stats_dict = None
contexto_historico = None
umbral_global = None
parametros_features = None

# Pool de threads para operaciones pesadas
executor = ThreadPoolExecutor(max_workers=4)

# ## 2. MODELOS PYDANTIC PARA VALIDACIÓN


In [7]:
class TipoAnomalia(str, Enum):
    MINUTOS_EXTREMOS = "MINUTOS_EXTREMOS"
    SPRAY_CALLING = "SPRAY_CALLING"
    VOLUMEN_ALTO = "VOLUMEN_ALTO"
    PAIS_BAJO_TRAFICO = "PAIS_BAJO_TRAFICO"
    NO_ANOMALIA = "NO_ANOMALIA"

class RegistroTelefonico(BaseModel):
    fecha: str = Field(..., description="Fecha en formato DD/MM/YYYY")
    codigodepais: int = Field(..., description="Código del país como número entero")  # Cambiado a int
    linea: str = Field(..., description="Número de línea telefónica")
    n_llamadas: int = Field(..., ge=0, description="Número de llamadas")
    n_minutos: float = Field(..., ge=0, description="Minutos totales")
    n_destinos: int = Field(..., ge=0, description="Número de destinos únicos")

class ResultadoScoring(BaseModel):
    fecha: str
    codigodepais: int  # Cambiado a int
    linea: str
    n_llamadas: int
    n_minutos: float
    n_destinos: int
    score_anomalia: float
    umbral: float
    es_anomalia: bool
    tipo_anomalia: TipoAnomalia
    tipo_contexto: str
    razon_decision: str
    timestamp_procesamiento: datetime

class EstadisticasModelo(BaseModel):
    fecha_entrenamiento: str
    fecha_ultima_actualizacion: Optional[str]
    umbral_global: float
    n_trees: int
    tree_height: int
    registros_entrenamiento: int
    paises_entrenamiento: int
    paises_conocidos: int
    version: str = "1.0"

class ConfiguracionUmbral(BaseModel):
    nuevo_umbral: float = Field(..., gt=0, le=1, description="Nuevo umbral entre 0 y 1")
    razon: str = Field(..., description="Razón del cambio")

class EstadoSistema(BaseModel):
    estado: str
    modelo_cargado: bool
    ultimo_procesamiento: Optional[datetime]
    registros_procesados_hoy: int
    anomalias_detectadas_hoy: int
    espacio_disco_gb: float
    version_api: str = "1.0"

# ## 3. FUNCIONES AUXILIARES DEL MODELO


In [8]:
def cargar_modelo():
    """Carga el modelo y configuración desde disco"""
    global modelo, scaler, config, stats_dict, contexto_historico, umbral_global, parametros_features
    
    try:
        # Cargar modelo
        with open(os.path.join(MODELS_PATH, "modelo_general.pkl"), 'rb') as f:
            modelo = pickle.load(f)
        
        # Cargar scaler
        with open(os.path.join(MODELS_PATH, "scaler_general.pkl"), 'rb') as f:
            scaler = pickle.load(f)
        
        # Cargar configuración
        with open(os.path.join(MODELS_PATH, "config_modelo_general.pkl"), 'rb') as f:
            config = pickle.load(f)
        
        # Extraer componentes
        stats_dict = config['stats_por_pais']
        contexto_historico = config.get('contexto_historico', {})
        umbral_global = config['umbral_global']
        parametros_features = config['parametros_features']
        
        logger.info("Modelo cargado exitosamente")
        return True
    except Exception as e:
        logger.error(f"Error al cargar modelo: {str(e)}")
        return False

def crear_features_contextualizadas_mejorada(row, stats_pais_dict):
    """Crea features para el modelo (idéntica a la original)"""
    pais = row['CODIGODEPAIS']
    llamadas = row['N_LLAMADAS']
    minutos = row['N_MINUTOS']
    destinos = row['N_DESTINOS']
    
    PESO_MINUTOS_NORMAL = parametros_features['peso_minutos_normal']
    PESO_MINUTOS_EXTREMOS = parametros_features['peso_minutos_extremos']
    UMBRAL_MINUTOS_EXTREMOS = parametros_features['umbral_minutos_extremos']
    PESO_DESTINOS = parametros_features['peso_destinos']
    PESO_SPRAY_RATIO = parametros_features['peso_spray_ratio']
    
    if pais in stats_pais_dict:
        pais_stats = stats_pais_dict[pais]
        categoria = pais_stats['CATEGORIA']
        
        llamadas_norm = min(llamadas / max(pais_stats['LLAMADAS_P95'], 1), 1.5)
        destinos_norm = min(destinos / max(pais_stats['DESTINOS_P95'], 1), 1.5)
        
        minutos_p90 = pais_stats.get('MINUTOS_P90', pais_stats['MINUTOS_P95'] * 0.9)
        
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            minutos_norm = min(minutos / max(minutos_p90, 1), 3.0)
            peso_minutos = PESO_MINUTOS_EXTREMOS
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(max(minutos_p90, 1)), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL
    else:
        categoria = 'Muy_Bajo'
        llamadas_norm = min(llamadas / 10, 2.0)
        destinos_norm = min(destinos / 5, 2.0)
        
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            minutos_norm = min(minutos / 50, 3.0)
            peso_minutos = PESO_MINUTOS_EXTREMOS * 1.2
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(60), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL
    
    features = {
        'llamadas_norm': llamadas_norm * 0.8,
        'destinos_norm': destinos_norm * PESO_DESTINOS,
        'minutos_norm': minutos_norm * peso_minutos,
        'diversidad_destinos': min(destinos / max(llamadas, 1), 1.0),
        'spray_ratio': min(destinos / max(llamadas, 1) * PESO_SPRAY_RATIO, 1.0) if destinos >= 5 else 0,
        'minutos_extremos': 1.0 if minutos >= UMBRAL_MINUTOS_EXTREMOS else 0.0,
        'minutos_sospechosos': min((minutos - 200) / 300, 1.0) if minutos > 200 else 0.0,
        'patron_spray_fuerte': 1.0 if (destinos >= 10 and llamadas >= 20) else 0.0,
        'patron_spray_medio': 0.5 if (destinos >= 6 and llamadas >= 12) else 0.0,
        'alta_diversidad': min(destinos / 12, 1) if destinos >= 5 else 0,
        'volumen_llamadas_alto': min((llamadas - 30) / 50, 1) if llamadas > 30 else 0,
        'volumen_destinos_alto': min((destinos - 10) / 20, 1) if destinos > 10 else 0,
        'llamadas_por_destino': min(llamadas / max(destinos, 1) / 5, 1),
        'eficiencia_destinos': min(destinos / max(llamadas * 0.5, 1), 1),
        'factor_pais_bajo': 1.5 if categoria in ['Muy_Bajo', 'Bajo'] else 1.0,
        'factor_pais_alto': 0.9 if categoria in ['Alto', 'Medio'] else 1.0
    }
    
    return features

def predecir_anomalia_individual(row_dict):
    """Realiza predicción para un registro individual"""
    if not modelo:
        raise ValueError("Modelo no cargado")
    
    # Extraer valores al inicio para usar en todo el ámbito
    # Convertir codigodepais a int si viene como string
    pais = int(row_dict['CODIGODEPAIS']) if isinstance(row_dict['CODIGODEPAIS'], str) else row_dict['CODIGODEPAIS']
    llamadas = row_dict['N_LLAMADAS']
    minutos = row_dict['N_MINUTOS']
    destinos = row_dict['N_DESTINOS']
    
    # Actualizar row_dict con el valor convertido
    row_dict['CODIGODEPAIS'] = pais
    
    # Crear features
    features = crear_features_contextualizadas_mejorada(row_dict, stats_dict)
    
    # Normalizar
    features_scaled = scaler.transform_one(features)
    
    # Obtener score
    score = modelo.score_one(features_scaled)
    
    # Lógica de decisión
    es_anomalia_base = score > umbral_global
    
    if es_anomalia_base:
        if minutos >= parametros_features['umbral_minutos_extremos']:
            es_anomalia_final = True
            razon = f"Minutos extremos ({minutos:.1f} min)"
            tipo_anomalia = TipoAnomalia.MINUTOS_EXTREMOS
        elif destinos >= 6 and llamadas >= 12:
            es_anomalia_final = True
            razon = "Patrón de spray calling confirmado"
            tipo_anomalia = TipoAnomalia.SPRAY_CALLING
        elif llamadas > 50 or destinos > 15:
            es_anomalia_final = True
            razon = "Volumen excepcionalmente alto"
            tipo_anomalia = TipoAnomalia.VOLUMEN_ALTO
        elif pais not in stats_dict or stats_dict.get(pais, {}).get('CATEGORIA') in ['Muy_Bajo', 'Bajo']:
            if destinos >= 4 and llamadas >= 8:
                es_anomalia_final = True
                razon = "Actividad sospechosa en país de bajo tráfico"
                tipo_anomalia = TipoAnomalia.PAIS_BAJO_TRAFICO
            else:
                es_anomalia_final = False
                razon = "Actividad baja en país de bajo tráfico"
                tipo_anomalia = TipoAnomalia.NO_ANOMALIA
        else:
            es_anomalia_final = False
            razon = "No cumple criterios de confirmación"
            tipo_anomalia = TipoAnomalia.NO_ANOMALIA
    else:
        es_anomalia_final = False
        razon = "Score bajo umbral"
        tipo_anomalia = TipoAnomalia.NO_ANOMALIA
    
    # Determinar contexto
    if contexto_historico and pais in contexto_historico:
        tipo_contexto = contexto_historico[pais]
    elif pais in stats_dict:
        tipo_contexto = stats_dict[pais]['CATEGORIA']
    else:
        tipo_contexto = "Muy_Bajo"
    
    return {
        'score': score,
        'umbral': umbral_global,
        'es_anomalia': es_anomalia_final,
        'tipo_anomalia': tipo_anomalia,
        'tipo_contexto': tipo_contexto,
        'razon_decision': razon
    }

# ## 4. INICIALIZACIÓN DE LA API


In [9]:
# Crear aplicación FastAPI
app = FastAPI(
    title="API de Detección de Fraude Telefónico",
    description="Sistema de detección de anomalías en llamadas telefónicas usando Machine Learning",
    version="1.0",
    docs_url="/docs",
    redoc_url="/redoc"
)

# Configurar CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.on_event("startup")
async def startup_event():
    """Carga el modelo e inicializa Oracle al iniciar la API"""
    logger.info("Iniciando API...")
    
    # Cargar modelo
    modelo_cargado = cargar_modelo()
    
    # Inicializar Oracle
    oracle_iniciado = False
    try:
        oracle_iniciado = inicializar_oracle_pool()
        if oracle_iniciado:
            crear_tablas_oracle()
    except Exception as e:
        logger.error(f"Error con Oracle: {e}")
        logger.info("La API funcionará sin Oracle")
    
    if modelo_cargado:
        logger.info(f"API iniciada - Modelo: OK, Oracle: {'OK' if oracle_iniciado else 'No disponible'}")
    else:
        logger.error("Error al iniciar API - Modelo no cargado")

@app.on_event("shutdown")
async def shutdown_event():
    """Cierra el pool de Oracle al detener la API"""
    global oracle_pool
    if oracle_pool:
        oracle_pool.close()
        logger.info("Pool de Oracle cerrado")


# ## 5. ENDPOINTS DE SCORING


In [10]:
@app.get("/", tags=["General"])
async def root():
    """Endpoint raíz con información de la API"""
    return {
        "mensaje": "API de Detección de Fraude Telefónico",
        "version": "1.0",
        "documentacion": "/docs",
        "estado": "activo" if modelo else "modelo no cargado"
    }

@app.post("/scoring/individual", response_model=ResultadoScoring, tags=["Scoring"])
async def scoring_individual(registro: RegistroTelefonico):
    """
    Realiza scoring de un registro individual
    """
    if not modelo:
        raise HTTPException(status_code=503, detail="Modelo no disponible")
    
    try:
        # Preparar datos
        row_dict = {
            'CODIGODEPAIS': registro.codigodepais,
            'N_LLAMADAS': registro.n_llamadas,
            'N_MINUTOS': registro.n_minutos,
            'N_DESTINOS': registro.n_destinos
        }
        
        # Realizar predicción
        resultado = predecir_anomalia_individual(row_dict)
        
        # Construir respuesta
        return ResultadoScoring(
            fecha=registro.fecha,
            codigodepais=registro.codigodepais,
            linea=registro.linea,
            n_llamadas=registro.n_llamadas,
            n_minutos=registro.n_minutos,
            n_destinos=registro.n_destinos,
            score_anomalia=round(resultado['score'], 4),
            umbral=round(resultado['umbral'], 4),
            es_anomalia=resultado['es_anomalia'],
            tipo_anomalia=resultado['tipo_anomalia'],
            tipo_contexto=resultado['tipo_contexto'],
            razon_decision=resultado['razon_decision'],
            timestamp_procesamiento=datetime.now()
        )
        
    except Exception as e:
        logger.error(f"Error en scoring individual: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/scoring/batch", tags=["Scoring"])
async def scoring_batch(file: UploadFile = File(...)):
    """
    Realiza scoring de un archivo CSV con múltiples registros
    y guarda anomalías en Oracle
    """
    if not modelo:
        raise HTTPException(status_code=503, detail="Modelo no disponible")
    
    lote_id = None
    temp_file = None
    
    try:
        # Guardar archivo temporal
        temp_file = os.path.join(TEMP_PATH, f"temp_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{file.filename}")
        
        with open(temp_file, "wb") as f:
            content = await file.read()
            f.write(content)
        
        # Leer CSV
        df = pd.read_csv(temp_file)
        
        # Convertir CODIGODEPAIS a int si es necesario
        if 'CODIGODEPAIS' in df.columns:
            df['CODIGODEPAIS'] = pd.to_numeric(df['CODIGODEPAIS'], errors='coerce').astype('Int64')
        
        # Validar columnas
        columnas_requeridas = ['FECHA', 'CODIGODEPAIS', 'LINEA', 'N_LLAMADAS', 'N_MINUTOS', 'N_DESTINOS']
        columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
        
        if columnas_faltantes:
            os.remove(temp_file)
            raise HTTPException(status_code=400, detail=f"Columnas faltantes: {columnas_faltantes}")
        
        # Procesar registros
        resultados = []
        anomalias_detectadas = 0
        
        for idx, row in df.iterrows():
            # Saltar filas con CODIGODEPAIS nulo
            if pd.isna(row['CODIGODEPAIS']):
                continue
                
            row_dict = {
                'CODIGODEPAIS': int(row['CODIGODEPAIS']),
                'N_LLAMADAS': row['N_LLAMADAS'],
                'N_MINUTOS': row['N_MINUTOS'],
                'N_DESTINOS': row['N_DESTINOS']
            }
            
            resultado = predecir_anomalia_individual(row_dict)
            
            resultado_completo = {
                'FECHA': row['FECHA'],
                'CODIGODEPAIS': int(row['CODIGODEPAIS']),
                'LINEA': row['LINEA'],
                'N_LLAMADAS': row['N_LLAMADAS'],
                'N_MINUTOS': row['N_MINUTOS'],
                'N_DESTINOS': row['N_DESTINOS'],
                'score_anomalia': round(resultado['score'], 4),
                'umbral': round(resultado['umbral'], 4),
                'es_anomalia': resultado['es_anomalia'],
                'tipo_anomalia': resultado['tipo_anomalia'].value,
                'tipo_contexto': resultado['tipo_contexto'],
                'razon_decision': resultado['razon_decision'],
                'timestamp_procesamiento': datetime.now()
            }
            
            resultados.append(resultado_completo)
            if resultado['es_anomalia']:
                anomalias_detectadas += 1
        
        # Guardar resultados en CSV
        df_resultados = pd.DataFrame(resultados)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        archivo_salida = os.path.join(OUTPUT_PATH, f"resultados_batch_{timestamp}.csv")
        df_resultados.to_csv(archivo_salida, index=False)
        
        # NUEVO: Guardar en Oracle si está disponible
        if oracle_pool:
            try:
                # Registrar lote
                lote_id = registrar_lote_procesamiento(
                    archivo_entrada=file.filename,
                    total_registros=len(resultados),
                    total_anomalias=anomalias_detectadas,
                    tasa_anomalias=round(anomalias_detectadas/len(resultados)*100, 2) if len(resultados) > 0 else 0,
                    archivo_salida=os.path.basename(archivo_salida)
                )
                
                # Guardar solo las anomalías
                df_anomalias = df_resultados[df_resultados['es_anomalia'] == True]
                if not df_anomalias.empty:
                    guardar_anomalias_oracle(df_anomalias, file.filename, str(lote_id))
                    
                logger.info(f"Datos guardados en Oracle - Lote ID: {lote_id}")
                
            except Exception as e:
                logger.error(f"Error guardando en Oracle: {e}")
                # No fallar el proceso si Oracle falla
        
        # Limpiar archivo temporal
        os.remove(temp_file)
        
        # Respuesta
        return {
            "mensaje": "Procesamiento completado",
            "registros_procesados": len(resultados),
            "anomalias_detectadas": anomalias_detectadas,
            "tasa_anomalias": round(anomalias_detectadas/len(resultados)*100, 2) if len(resultados) > 0 else 0,
            "archivo_resultados": os.path.basename(archivo_salida),
            "lote_id": lote_id,
            "timestamp": datetime.now(),
            "guardado_oracle": oracle_pool is not None
        }
        
    except Exception as e:
        logger.error(f"Error en scoring batch: {str(e)}")
        if temp_file and os.path.exists(temp_file):
            os.remove(temp_file)
        
        # Registrar error en Oracle si es posible
        if oracle_pool and lote_id is None:
            try:
                with get_oracle_connection() as conn:
                    cursor = conn.cursor()
                    cursor.execute("""
                        INSERT INTO LOTES_PROCESAMIENTO 
                        (TIMESTAMP_INICIO, ARCHIVO_ENTRADA, ESTADO, MENSAJE_ERROR)
                        VALUES (:1, :2, :3, :4)
                    """, [datetime.now(), file.filename, "ERROR", str(e)])
                    conn.commit()
            except:
                pass
                
        raise HTTPException(status_code=500, detail=str(e))

# ## NUEVO ENDPOINT CONSULTAS ORACLE

In [11]:
@app.get("/alertas/consultar", tags=["Alertas Oracle"])
async def consultar_alertas(
    fecha_inicio: Optional[str] = Query(None, description="Fecha inicio YYYY-MM-DD"),
    fecha_fin: Optional[str] = Query(None, description="Fecha fin YYYY-MM-DD"),
    codigo_pais: Optional[int] = Query(None, description="Código de país"),
    tipo_anomalia: Optional[str] = Query(None, description="Tipo de anomalía"),
    limite: int = Query(100, description="Número máximo de registros")
):
    """
    Consulta las alertas guardadas en Oracle con filtros opcionales
    """
    if not oracle_pool:
        raise HTTPException(status_code=503, detail="Oracle no disponible")
    
    try:
        with get_oracle_connection() as conn:
            cursor = conn.cursor()
            
            # Construir query dinámicamente
            query = """
                SELECT ID_ALERTA, FECHA_PROCESAMIENTO, FECHA_REGISTRO, CODIGO_PAIS, 
                       LINEA, N_LLAMADAS, N_MINUTOS, N_DESTINOS, SCORE_ANOMALIA,
                       UMBRAL, TIPO_ANOMALIA, TIPO_CONTEXTO, RAZON_DECISION
                FROM ALERTAS_FRAUDE
                WHERE 1=1
            """
            params = []
            
            if fecha_inicio:
                query += " AND FECHA_PROCESAMIENTO >= TO_DATE(:fecha_inicio, 'YYYY-MM-DD')"
                params.append(fecha_inicio)
            
            if fecha_fin:
                query += " AND FECHA_PROCESAMIENTO <= TO_DATE(:fecha_fin, 'YYYY-MM-DD') + 1"
                params.append(fecha_fin)
            
            if codigo_pais:
                query += " AND CODIGO_PAIS = :codigo_pais"
                params.append(codigo_pais)
            
            if tipo_anomalia:
                query += " AND TIPO_ANOMALIA = :tipo_anomalia"
                params.append(tipo_anomalia)
            
            query += " ORDER BY FECHA_PROCESAMIENTO DESC"
            query += f" FETCH FIRST {limite} ROWS ONLY"
            
            cursor.execute(query, params)
            
            # Obtener nombres de columnas
            columnas = [col[0] for col in cursor.description]
            
            # Convertir a lista de diccionarios
            alertas = []
            for row in cursor:
                alerta = dict(zip(columnas, row))
                # Convertir datetime a string
                if alerta['FECHA_PROCESAMIENTO']:
                    alerta['FECHA_PROCESAMIENTO'] = alerta['FECHA_PROCESAMIENTO'].isoformat()
                alertas.append(alerta)
            
            return {
                "total": len(alertas),
                "alertas": alertas
            }
            
    except Exception as e:
        logger.error(f"Error consultando alertas: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/alertas/estadisticas", tags=["Alertas Oracle"])
async def estadisticas_alertas(
    fecha_inicio: str = Query(..., description="Fecha inicio YYYY-MM-DD"),
    fecha_fin: str = Query(..., description="Fecha fin YYYY-MM-DD")
):
    """
    Obtiene estadísticas de las alertas en un rango de fechas
    """
    if not oracle_pool:
        raise HTTPException(status_code=503, detail="Oracle no disponible")
    
    try:
        with get_oracle_connection() as conn:
            cursor = conn.cursor()
            
            # Estadísticas generales
            cursor.execute("""
                SELECT 
                    COUNT(*) as total_alertas,
                    COUNT(DISTINCT CODIGO_PAIS) as paises_afectados,
                    COUNT(DISTINCT LINEA) as lineas_afectadas,
                    AVG(SCORE_ANOMALIA) as score_promedio,
                    MAX(SCORE_ANOMALIA) as score_maximo
                FROM ALERTAS_FRAUDE
                WHERE FECHA_PROCESAMIENTO >= TO_DATE(:1, 'YYYY-MM-DD')
                  AND FECHA_PROCESAMIENTO <= TO_DATE(:2, 'YYYY-MM-DD') + 1
            """, [fecha_inicio, fecha_fin])
            
            stats_generales = dict(zip(
                ['total_alertas', 'paises_afectados', 'lineas_afectadas', 
                 'score_promedio', 'score_maximo'],
                cursor.fetchone()
            ))
            
            # Distribución por tipo
            cursor.execute("""
                SELECT TIPO_ANOMALIA, COUNT(*) as cantidad
                FROM ALERTAS_FRAUDE
                WHERE FECHA_PROCESAMIENTO >= TO_DATE(:1, 'YYYY-MM-DD')
                  AND FECHA_PROCESAMIENTO <= TO_DATE(:2, 'YYYY-MM-DD') + 1
                GROUP BY TIPO_ANOMALIA
                ORDER BY cantidad DESC
            """, [fecha_inicio, fecha_fin])
            
            distribucion_tipos = [
                {"tipo": row[0], "cantidad": row[1]}
                for row in cursor.fetchall()
            ]
            
            # Top países
            cursor.execute("""
                SELECT CODIGO_PAIS, COUNT(*) as cantidad
                FROM ALERTAS_FRAUDE
                WHERE FECHA_PROCESAMIENTO >= TO_DATE(:1, 'YYYY-MM-DD')
                  AND FECHA_PROCESAMIENTO <= TO_DATE(:2, 'YYYY-MM-DD') + 1
                GROUP BY CODIGO_PAIS
                ORDER BY cantidad DESC
                FETCH FIRST 10 ROWS ONLY
            """, [fecha_inicio, fecha_fin])
            
            top_paises = [
                {"codigo_pais": row[0], "cantidad": row[1]}
                for row in cursor.fetchall()
            ]
            
            # Tendencia diaria
            cursor.execute("""
                SELECT 
                    TO_CHAR(FECHA_PROCESAMIENTO, 'YYYY-MM-DD') as fecha,
                    COUNT(*) as cantidad
                FROM ALERTAS_FRAUDE
                WHERE FECHA_PROCESAMIENTO >= TO_DATE(:1, 'YYYY-MM-DD')
                  AND FECHA_PROCESAMIENTO <= TO_DATE(:2, 'YYYY-MM-DD') + 1
                GROUP BY TO_CHAR(FECHA_PROCESAMIENTO, 'YYYY-MM-DD')
                ORDER BY fecha
            """, [fecha_inicio, fecha_fin])
            
            tendencia_diaria = [
                {"fecha": row[0], "cantidad": row[1]}
                for row in cursor.fetchall()
            ]
            
            return {
                "periodo": {
                    "fecha_inicio": fecha_inicio,
                    "fecha_fin": fecha_fin
                },
                "estadisticas_generales": stats_generales,
                "distribucion_tipos": distribucion_tipos,
                "top_paises": top_paises,
                "tendencia_diaria": tendencia_diaria
            }
            
    except Exception as e:
        logger.error(f"Error obteniendo estadísticas: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/lotes/historial", tags=["Alertas Oracle"])
async def historial_lotes(limite: int = Query(50, description="Número de lotes a mostrar")):
    """
    Obtiene el historial de lotes procesados
    """
    if not oracle_pool:
        raise HTTPException(status_code=503, detail="Oracle no disponible")
    
    try:
        with get_oracle_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT ID_LOTE, TIMESTAMP_INICIO, TIMESTAMP_FIN, ARCHIVO_ENTRADA,
                       TOTAL_REGISTROS, TOTAL_ANOMALIAS, TASA_ANOMALIAS,
                       ARCHIVO_SALIDA, ESTADO
                FROM LOTES_PROCESAMIENTO
                ORDER BY TIMESTAMP_INICIO DESC
                FETCH FIRST :limite ROWS ONLY
            """, [limite])
            
            columnas = [col[0] for col in cursor.description]
            lotes = []
            
            for row in cursor:
                lote = dict(zip(columnas, row))
                # Convertir timestamps
                if lote['TIMESTAMP_INICIO']:
                    lote['TIMESTAMP_INICIO'] = lote['TIMESTAMP_INICIO'].isoformat()
                if lote['TIMESTAMP_FIN']:
                    lote['TIMESTAMP_FIN'] = lote['TIMESTAMP_FIN'].isoformat()
                lotes.append(lote)
            
            return {
                "total": len(lotes),
                "lotes": lotes
            }
            
    except Exception as e:
        logger.error(f"Error obteniendo historial de lotes: {e}")
        raise HTTPException(status_code=500, detail=str(e))

# ##FUNCIONES PARA APRENDISAJE INCREMENTAL / ORACLE

In [12]:
def determinar_categoria_pais(stats):
    """Determina la categoría de un país basado en sus estadísticas"""
    # Basado en el promedio de llamadas
    llamadas_mean = stats.get('LLAMADAS_MEAN', 0)
    
    if llamadas_mean >= 50:
        return 'Alto'
    elif llamadas_mean >= 20:
        return 'Medio'
    elif llamadas_mean >= 10:
        return 'Bajo'
    else:
        return 'Muy_Bajo'

def calcular_umbral_adaptativo(modelo, scaler, df_muestra, stats_dict):
    """Calcula un umbral adaptativo basado en los scores actuales"""
    scores = []
    
    # Tomar una muestra aleatoria
    muestra = df_muestra.sample(min(1000, len(df_muestra)))
    
    for _, row in muestra.iterrows():
        row_dict = {
            'CODIGODEPAIS': int(row['CODIGODEPAIS']) if not pd.isna(row['CODIGODEPAIS']) else 0,
            'N_LLAMADAS': row['N_LLAMADAS'],
            'N_MINUTOS': row['N_MINUTOS'],
            'N_DESTINOS': row['N_DESTINOS']
        }
        
        features = crear_features_contextualizadas_mejorada(row_dict, stats_dict)
        features_scaled = scaler.transform_one(features)
        score = modelo.score_one(features_scaled)
        scores.append(score)
    
    # Nuevo umbral = percentil 95 de los scores
    return np.percentile(scores, 95)

def registrar_inicio_aprendizaje(archivo_path, total_registros, estado_inicial):
    """Registra el inicio del aprendizaje incremental en Oracle"""
    with get_oracle_connection() as conn:
        cursor = conn.cursor()
        
        # Contar países únicos en el archivo
        try:
            df_temp = pd.read_csv(archivo_path)
            paises_en_archivo = df_temp['CODIGODEPAIS'].nunique()
        except:
            paises_en_archivo = 0
        
        id_var = cursor.var(oracledb.NUMBER)
        
        cursor.execute("""
            INSERT INTO APRENDIZAJE_INCREMENTAL (
                FECHA_DATOS, ARCHIVO_ENTRADA, TOTAL_REGISTROS,
                PAISES_TOTALES, UMBRAL_ANTERIOR, N_TREES_ANTERIOR,
                PAISES_CONOCIDOS_ANTERIOR, ESTADO
            ) VALUES (
                TRUNC(SYSDATE), :1, :2, :3, :4, :5, :6, 'EN_PROCESO'
            ) RETURNING ID_APRENDIZAJE INTO :id
        """, [
            os.path.basename(archivo_path),
            total_registros,
            paises_en_archivo,
            estado_inicial['umbral'],
            estado_inicial['n_trees'],
            estado_inicial['paises_conocidos'],
            id_var
        ])
        
        conn.commit()
        return id_var.getvalue()[0]

def actualizar_fin_aprendizaje(id_aprendizaje, **kwargs):
    """Actualiza el registro al finalizar el aprendizaje"""
    with get_oracle_connection() as conn:
        cursor = conn.cursor()
        
        # Preparar JSON de cambios de categoría
        cambios_json = json.dumps(kwargs.get('cambios_categoria', {}))
        
        cursor.execute("""
            UPDATE APRENDIZAJE_INCREMENTAL SET
                REGISTROS_PROCESADOS = :1,
                PAISES_NUEVOS = :2,
                PAISES_ACTUALIZADOS = :3,
                UMBRAL_NUEVO = :4,
                N_TREES_NUEVO = :5,
                PAISES_CONOCIDOS_NUEVO = :6,
                PAISES_CON_CAMBIO_CATEGORIA = :7,
                DETALLE_CAMBIOS_CATEGORIA = :8,
                TIEMPO_PROCESAMIENTO_SEG = :9,
                ESTADO = :10,
                ARCHIVO_BACKUP_CONFIG = :11,
                LLAMADAS_PROMEDIO_GLOBAL = :12,
                MINUTOS_PROMEDIO_GLOBAL = :13,
                DESTINOS_PROMEDIO_GLOBAL = :14,
                UMBRAL_MINUTOS_EXTREMOS = :15,
                PESO_SPRAY_RATIO = :16
            WHERE ID_APRENDIZAJE = :17
        """, [
            kwargs['registros_procesados'],
            kwargs['paises_nuevos'],
            kwargs['paises_actualizados'],
            kwargs['umbral_nuevo'],
            config.get('n_trees', 0),
            len(stats_dict),
            len(kwargs.get('cambios_categoria', {})),
            cambios_json,
            kwargs['tiempo_procesamiento'],
            kwargs['estado'],
            os.path.basename(kwargs['backup_path']),
            kwargs['promedios_globales']['llamadas'],
            kwargs['promedios_globales']['minutos'],
            kwargs['promedios_globales']['destinos'],
            parametros_features.get('umbral_minutos_extremos', 0),
            parametros_features.get('peso_spray_ratio', 0),
            id_aprendizaje
        ])
        
        conn.commit()

# ## 6. ENDPOINTS DE APRENDIZAJE INCREMENTAL


In [13]:
@app.post("/modelo/aprendizaje-incremental", tags=["Modelo"])
async def aprendizaje_incremental(
    file: UploadFile = File(...),
    background_tasks: BackgroundTasks = BackgroundTasks()
):
    """
    Realiza aprendizaje incremental con nuevos datos
    """
    if not modelo:
        raise HTTPException(status_code=503, detail="Modelo no disponible")
    
    try:
        # Guardar archivo temporal
        temp_file = os.path.join(TEMP_PATH, f"incremental_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{file.filename}")
        
        with open(temp_file, "wb") as f:
            content = await file.read()
            f.write(content)
        
        # Validar archivo
        df = pd.read_csv(temp_file)
        
        # Agregar tarea en background
        background_tasks.add_task(procesar_aprendizaje_incremental, temp_file, df)
        
        return {
            "mensaje": "Aprendizaje incremental iniciado en background",
            "registros_a_procesar": len(df),
            "timestamp": datetime.now()
        }
        
    except Exception as e:
        logger.error(f"Error iniciando aprendizaje incremental: {str(e)}")
        if os.path.exists(temp_file):
            os.remove(temp_file)
        raise HTTPException(status_code=500, detail=str(e))

def procesar_aprendizaje_incremental(archivo_path: str, df: pd.DataFrame):
    """Procesa el aprendizaje incremental en background y registra en Oracle"""
    global stats_dict, umbral_global
    
    # Variables para tracking
    tiempo_inicio = datetime.now()
    id_aprendizaje = None
    estado_inicial = {
        'umbral': umbral_global,
        'n_trees': config.get('n_trees', 0),
        'paises_conocidos': len(stats_dict),
        'categorias': {pais: stats.get('CATEGORIA', 'Desconocido') 
                      for pais, stats in stats_dict.items()}
    }
    
    try:
        logger.info(f"Iniciando aprendizaje incremental con {len(df)} registros")
        
        # Registrar inicio en Oracle
        if oracle_pool:
            id_aprendizaje = registrar_inicio_aprendizaje(
                archivo_path, len(df), estado_inicial
            )
        
        registros_procesados = 0
        paises_nuevos = set()
        paises_actualizados = set()
        cambios_categoria = {}
        
        # Procesar cada registro
        for idx, row in df.iterrows():
            row_dict = {
                'CODIGODEPAIS': int(row['CODIGODEPAIS']) if not pd.isna(row['CODIGODEPAIS']) else 0,
                'N_LLAMADAS': row['N_LLAMADAS'],
                'N_MINUTOS': row['N_MINUTOS'],
                'N_DESTINOS': row['N_DESTINOS']
            }
            
            # Crear features
            features = crear_features_contextualizadas_mejorada(row_dict, stats_dict)
            
            # Actualizar scaler
            scaler.learn_one(features)
            features_scaled = scaler.transform_one(features)
            
            # Aprender
            modelo.learn_one(features_scaled)
            
            # Detectar países nuevos
            pais = row_dict['CODIGODEPAIS']
            if pais not in stats_dict:
                paises_nuevos.add(pais)
            
            registros_procesados += 1
        
        # Actualizar estadísticas por país
        estadisticas_globales = {
            'llamadas_total': 0,
            'minutos_total': 0,
            'destinos_total': 0,
            'registros_total': 0
        }
        
        for pais in df['CODIGODEPAIS'].unique():
            if pd.isna(pais):
                continue
                
            pais = int(pais)
            datos_pais = df[df['CODIGODEPAIS'] == pais]
            
            if len(datos_pais) >= 20:  # Mínimo 20 registros
                categoria_anterior = stats_dict.get(pais, {}).get('CATEGORIA', None)
                
                stats_nuevas = {
                    'REGISTROS': len(datos_pais),
                    'LLAMADAS_MEAN': datos_pais['N_LLAMADAS'].mean(),
                    'LLAMADAS_STD': datos_pais['N_LLAMADAS'].std(),
                    'LLAMADAS_P90': datos_pais['N_LLAMADAS'].quantile(0.90),
                    'LLAMADAS_P95': datos_pais['N_LLAMADAS'].quantile(0.95),
                    'MINUTOS_MEAN': datos_pais['N_MINUTOS'].mean(),
                    'MINUTOS_STD': datos_pais['N_MINUTOS'].std(),
                    'MINUTOS_P90': datos_pais['N_MINUTOS'].quantile(0.90),
                    'MINUTOS_P95': datos_pais['N_MINUTOS'].quantile(0.95),
                    'DESTINOS_MEAN': datos_pais['N_DESTINOS'].mean(),
                    'DESTINOS_STD': datos_pais['N_DESTINOS'].std(),
                    'DESTINOS_P90': datos_pais['N_DESTINOS'].quantile(0.90),
                    'DESTINOS_P95': datos_pais['N_DESTINOS'].quantile(0.95),
                }
                
                # Actualizar estadísticas globales
                estadisticas_globales['llamadas_total'] += datos_pais['N_LLAMADAS'].sum()
                estadisticas_globales['minutos_total'] += datos_pais['N_MINUTOS'].sum()
                estadisticas_globales['destinos_total'] += datos_pais['N_DESTINOS'].sum()
                estadisticas_globales['registros_total'] += len(datos_pais)
                
                # Determinar nueva categoría basada en volumen
                nueva_categoria = determinar_categoria_pais(stats_nuevas)
                stats_nuevas['CATEGORIA'] = nueva_categoria
                
                # Actualizar o agregar país
                if pais in stats_dict:
                    # Promediar con existente
                    for campo in ['LLAMADAS_MEAN', 'MINUTOS_MEAN', 'DESTINOS_MEAN']:
                        if campo in stats_dict[pais]:
                            stats_dict[pais][campo] = (stats_dict[pais][campo] * 0.7 + 
                                                      stats_nuevas[campo] * 0.3)
                    
                    # Actualizar percentiles y categoría
                    for campo in ['LLAMADAS_P90', 'LLAMADAS_P95', 'MINUTOS_P90', 'MINUTOS_P95', 
                                 'DESTINOS_P90', 'DESTINOS_P95', 'CATEGORIA']:
                        stats_dict[pais][campo] = stats_nuevas[campo]
                    
                    paises_actualizados.add(pais)
                    
                    # Detectar cambio de categoría
                    if categoria_anterior and categoria_anterior != nueva_categoria:
                        cambios_categoria[pais] = {
                            'anterior': categoria_anterior,
                            'nueva': nueva_categoria
                        }
                else:
                    # País nuevo
                    stats_dict[pais] = stats_nuevas
        
        # Calcular promedios globales
        if estadisticas_globales['registros_total'] > 0:
            promedios_globales = {
                'llamadas': estadisticas_globales['llamadas_total'] / estadisticas_globales['registros_total'],
                'minutos': estadisticas_globales['minutos_total'] / estadisticas_globales['registros_total'],
                'destinos': estadisticas_globales['destinos_total'] / estadisticas_globales['registros_total']
            }
        else:
            promedios_globales = {'llamadas': 0, 'minutos': 0, 'destinos': 0}
        
        # Recalcular umbral adaptativo si hay suficientes datos
        if registros_procesados > 1000:
            umbral_global = calcular_umbral_adaptativo(modelo, scaler, df, stats_dict)
            config['umbral_global'] = umbral_global
        
        # Actualizar configuración
        config['stats_por_pais'] = stats_dict
        config['fecha_ultima_actualizacion'] = datetime.now().isoformat()
        config['ultimo_aprendizaje_incremental'] = {
            'fecha': datetime.now().isoformat(),
            'registros_procesados': registros_procesados,
            'paises_nuevos': len(paises_nuevos),
            'paises_actualizados': len(paises_actualizados)
        }
        
        # Guardar modelo actualizado
        with open(os.path.join(MODELS_PATH, "modelo_general.pkl"), 'wb') as f:
            pickle.dump(modelo, f)
        
        with open(os.path.join(MODELS_PATH, "scaler_general.pkl"), 'wb') as f:
            pickle.dump(scaler, f)
        
        with open(os.path.join(MODELS_PATH, "config_modelo_general.pkl"), 'wb') as f:
            pickle.dump(config, f)
        
        # Crear backup con timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_path = os.path.join(MODELS_PATH, f"backup_config_{timestamp}.pkl")
        with open(backup_path, 'wb') as f:
            pickle.dump(config, f)
        
        # Calcular tiempo de procesamiento
        tiempo_fin = datetime.now()
        tiempo_procesamiento = (tiempo_fin - tiempo_inicio).total_seconds()
        
        # Actualizar registro en Oracle
        if oracle_pool and id_aprendizaje:
            actualizar_fin_aprendizaje(
                id_aprendizaje=id_aprendizaje,
                registros_procesados=registros_procesados,
                paises_nuevos=len(paises_nuevos),
                paises_actualizados=len(paises_actualizados),
                umbral_nuevo=umbral_global,
                cambios_categoria=cambios_categoria,
                tiempo_procesamiento=tiempo_procesamiento,
                backup_path=backup_path,
                promedios_globales=promedios_globales,
                estado="COMPLETADO"
            )
        
        # Limpiar archivo temporal
        os.remove(archivo_path)
        
        logger.info(f"""
        Aprendizaje incremental completado:
        - Registros procesados: {registros_procesados}
        - Países nuevos: {len(paises_nuevos)}
        - Países actualizados: {len(paises_actualizados)}
        - Cambios de categoría: {len(cambios_categoria)}
        - Nuevo umbral: {umbral_global:.4f}
        - Tiempo: {tiempo_procesamiento:.2f} segundos
        """)
        
    except Exception as e:
        logger.error(f"Error en aprendizaje incremental: {str(e)}")
        
        # Actualizar registro de error en Oracle
        if oracle_pool and id_aprendizaje:
            try:
                with get_oracle_connection() as conn:
                    cursor = conn.cursor()
                    cursor.execute("""
                        UPDATE APRENDIZAJE_INCREMENTAL
                        SET ESTADO = 'ERROR',
                            MENSAJE_ERROR = :1,
                            TIEMPO_PROCESAMIENTO_SEG = :2
                        WHERE ID_APRENDIZAJE = :3
                    """, [str(e), (datetime.now() - tiempo_inicio).total_seconds(), id_aprendizaje])
                    conn.commit()
            except:
                pass
        
        if os.path.exists(archivo_path):
            os.remove(archivo_path)


# ## 7. ENDPOINTS DE GESTIÓN DEL MODELO


In [14]:
@app.get("/modelo/estadisticas", response_model=EstadisticasModelo, tags=["Modelo"])
async def obtener_estadisticas_modelo():
    """
    Obtiene las estadísticas actuales del modelo
    """
    if not modelo or not config:
        raise HTTPException(status_code=503, detail="Modelo no disponible")

    return EstadisticasModelo(
        fecha_entrenamiento=config.get('fecha_entrenamiento', 'No disponible'),
        fecha_ultima_actualizacion=config.get('fecha_ultima_actualizacion'),
        umbral_global=umbral_global,
        n_trees=config.get('n_trees', 0),
        tree_height=config.get('tree_height', 0),
        registros_entrenamiento=config.get('registros_entrenamiento', 0),
        paises_entrenamiento=config.get('paises_entrenamiento', 0),
        paises_conocidos=len(stats_dict)
    )

@app.put("/modelo/actualizar-umbral", tags=["Modelo"])
async def actualizar_umbral(configuracion: ConfiguracionUmbral):
    """
    Actualiza el umbral de detección de anomalías
    """
    global umbral_global

    if not modelo:
        raise HTTPException(status_code=503, detail="Modelo no disponible")

    try:
        # Guardar umbral anterior
        umbral_anterior = umbral_global

        # Actualizar umbral
        umbral_global = configuracion.nuevo_umbral
        config['umbral_global'] = umbral_global

        # Guardar configuración actualizada
        with open(os.path.join(MODELS_PATH, "config_modelo_general.pkl"), 'wb') as f:
            pickle.dump(config, f)

        # NUEVO: Guardar en Oracle si está disponible
        if oracle_pool:
            try:
                with get_oracle_connection() as conn:
                    cursor = conn.cursor()
                    cursor.execute("""
                        INSERT INTO HISTORICO_UMBRALES 
                        (UMBRAL_ANTERIOR, UMBRAL_NUEVO, RAZON_CAMBIO, USUARIO)
                        VALUES (:1, :2, :3, :4)
                    """, [umbral_anterior, umbral_global, configuracion.razon, "API"])
                    conn.commit()
                    logger.info("Cambio de umbral registrado en Oracle")
            except Exception as e:
                logger.error(f"Error guardando cambio de umbral en Oracle: {e}")

        # Guardar también en archivo JSON (como respaldo)
        log_cambio = {
            'timestamp': datetime.now(),
            'umbral_anterior': umbral_anterior,
            'umbral_nuevo': umbral_global,
            'razon': configuracion.razon
        }
        log_file = os.path.join(LOG_PATH, f"cambios_umbral_{datetime.now():%Y%m}.json")

        if os.path.exists(log_file):
            with open(log_file, 'r') as f:
                logs = json.load(f)
        else:
            logs = []

        logs.append(log_cambio)
        with open(log_file, 'w') as f:
            json.dump(logs, f, indent=2, default=str)

        return {
            "mensaje": "Umbral actualizado exitosamente",
            "umbral_anterior": umbral_anterior,
            "umbral_nuevo": umbral_global,
            "timestamp": datetime.now(),
            "guardado_oracle": oracle_pool is not None
        }

    except Exception as e:
        logger.error(f"Error actualizando umbral: {e}")
        raise HTTPException(status_code=500, detail=str(e))


@app.post("/modelo/recargar", tags=["Modelo"])
async def recargar_modelo():
    """
    Recarga el modelo desde disco (útil si se actualizó externamente)
    """
    try:
        if cargar_modelo():
            return {
                "mensaje": "Modelo recargado exitosamente",
                "timestamp": datetime.now()
            }
        else:
            raise HTTPException(status_code=500, detail="Error al recargar modelo")
    except Exception as e:
        logger.error(f"Error recargando modelo: {e}")
        raise HTTPException(status_code=500, detail=str(e))

## 8. ENDPOINTS DE ANÁLISIS Y REPORTES

In [15]:


@app.get("/analisis/resumen-diario", tags=["Análisis"])
async def resumen_diario(fecha: str = Query(..., description="Fecha en formato YYYY-MM-DD")):
    """
    Obtiene un resumen de las anomalías detectadas en un día específico
    """
    try:
        # Convertir fecha
        fecha_obj = datetime.strptime(fecha, "%Y-%m-%d")

        # Buscar archivos de resultados del día
        archivos_dia = [
            f for f in os.listdir(OUTPUT_PATH)
            if f.startswith("resultados_") and fecha.replace("-", "") in f
        ]

        if not archivos_dia:
            return {
                "fecha": fecha,
                "registros_procesados": 0,
                "anomalias_detectadas": 0,
                "mensaje": "No hay datos para esta fecha"
            }

        # Cargar y combinar datos
        dfs = [pd.read_csv(os.path.join(OUTPUT_PATH, f)) for f in archivos_dia]
        df_total = pd.concat(dfs, ignore_index=True)

        # Calcular estadísticas
        total_registros = len(df_total)
        total_anomalias = int(df_total['es_anomalia'].sum())

        # Distribución por tipo
        distribucion_tipos = {}
        anomalias = df_total[df_total['es_anomalia'] == True]
        for tipo, cnt in anomalias['tipo_anomalia'].value_counts().items():
            distribucion_tipos[tipo] = {
                "cantidad": int(cnt),
                "porcentaje": round(cnt / len(anomalias) * 100, 2)
            }

        # Top países
        top_paises = [
            {"pais": pais, "anomalias": int(cnt)}
            for pais, cnt in anomalias['CODIGODEPAIS'].value_counts().head(10).items()
        ]

        return {
            "fecha": fecha,
            "registros_procesados": total_registros,
            "anomalias_detectadas": total_anomalias,
            "tasa_anomalias": round(total_anomalias / total_registros * 100, 2) if total_registros > 0 else 0,
            "distribucion_tipos": distribucion_tipos,
            "top_paises_anomalias": top_paises
        }

    except Exception as e:
        logger.error(f"Error generando resumen diario: {e}")
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/analisis/tendencias", tags=["Análisis"])
async def obtener_tendencias(dias: int = Query(7, description="Número de días a analizar")):
    """
    Obtiene las tendencias de anomalías de los últimos N días
    """
    try:
        fecha_fin = datetime.now()
        fecha_inicio = fecha_fin - timedelta(days=dias)
        tendencias = []

        for i in range(dias):
            fecha_i = fecha_inicio + timedelta(days=i)
            archivos = [
                f for f in os.listdir(OUTPUT_PATH)
                if f.startswith("resultados_") and fecha_i.strftime("%Y%m%d") in f
            ]

            if archivos:
                dfs = [pd.read_csv(os.path.join(OUTPUT_PATH, f)) for f in archivos]
                df_dia = pd.concat(dfs, ignore_index=True)
                anom_count = int(df_dia['es_anomalia'].sum())
                tasas = round(df_dia['es_anomalia'].mean() * 100, 2) if len(df_dia) > 0 else 0
                registros = len(df_dia)
            else:
                anom_count = 0
                tasas = 0
                registros = 0

            tendencias.append({
                "fecha": fecha_i.strftime("%Y-%m-%d"),
                "registros": registros,
                "anomalias": anom_count,
                "tasa": tasas
            })

        registros_totales = sum(t['registros'] for t in tendencias)
        anom_totales = sum(t['anomalias'] for t in tendencias)

        return {
            "periodo": f"Últimos {dias} días",
            "fecha_inicio": fecha_inicio.strftime("%Y-%m-%d"),
            "fecha_fin": fecha_fin.strftime("%Y-%m-%d"),
            "tendencias": tendencias,
            "resumen": {
                "registros_totales": registros_totales,
                "anomalias_totales": anom_totales,
                "tasa_promedio": round(anom_totales / registros_totales * 100, 2) if registros_totales > 0 else 0
            }
        }

    except Exception as e:
        logger.error(f"Error obteniendo tendencias: {e}")
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/analisis/exportar-anomalias", tags=["Análisis"])
async def exportar_anomalias(
    fecha_inicio: str = Query(..., description="Fecha inicio YYYY-MM-DD"),
    fecha_fin: str    = Query(..., description="Fecha fin YYYY-MM-DD")
):
    """
    Exporta todas las anomalías detectadas en un rango de fechas
    """
    try:
        # Convertir fechas
        inicio = datetime.strptime(fecha_inicio, "%Y-%m-%d")
        fin    = datetime.strptime(fecha_fin,    "%Y-%m-%d")

        if inicio > fin:
            raise HTTPException(status_code=400, detail="Fecha inicio debe ser anterior a fecha fin")

        anomalias_totales = []
        for archivo in os.listdir(OUTPUT_PATH):
            if archivo.startswith("resultados_"):
                fecha_arch = archivo.split("_")[2][:8]
                try:
                    fecha_obj = datetime.strptime(fecha_arch, "%Y%m%d")
                except ValueError:
                    continue

                if inicio <= fecha_obj <= fin:
                    df = pd.read_csv(os.path.join(OUTPUT_PATH, archivo))
                    anomalias_totales.append(df[df['es_anomalia'] == True])

        if not anomalias_totales:
            return {
                "mensaje": "No se encontraron anomalías en el rango especificado",
                "fecha_inicio": fecha_inicio,
                "fecha_fin": fecha_fin
            }

        # Combinar y exportar
        df_out = pd.concat(anomalias_totales, ignore_index=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join(OUTPUT_PATH, f"exportacion_anomalias_{timestamp}.csv")
        df_out.to_csv(out_path, index=False)

        return FileResponse(out_path, media_type="text/csv",
                            filename=f"anomalias_{fecha_inicio}_{fecha_fin}.csv")

    except Exception as e:
        logger.error(f"Error exportando anomalías: {e}")
        raise HTTPException(status_code=500, detail=str(e))


## 9. ENDPOINTS DE MONITOREO Y SALUD

In [16]:
@app.get("/salud", response_model=EstadoSistema, tags=["Monitoreo"])
async def verificar_salud():
    """
    Verifica el estado de salud del sistema
    """
    try:
        # Calcular estadísticas del día
        hoy = datetime.now().strftime("%Y%m%d")
        registros_hoy = 0
        anomalias_hoy = 0
        for archivo in os.listdir(OUTPUT_PATH):
            if archivo.startswith("resultados_") and hoy in archivo:
                df = pd.read_csv(os.path.join(OUTPUT_PATH, archivo))
                registros_hoy += len(df)
                anomalias_hoy += int(df['es_anomalia'].sum())

        # Espacio en disco
        stat = shutil.disk_usage(OUTPUT_PATH)
        espacio_gb = stat.free / (1024**3)

        # Último procesamiento
        archivos = [f for f in os.listdir(OUTPUT_PATH) if f.startswith("resultados_")]
        ultimo_procesamiento = None
        if archivos:
            ultimo_archivo = max(archivos)
            # Extraer timestamp del nombre del archivo
            try:
                parts = ultimo_archivo.split("_")
                timestamp_str = parts[2] + parts[3].replace(".csv", "")
                ultimo_procesamiento = datetime.strptime(timestamp_str, "%Y%m%d%H%M%S")
            except:
                pass

        return EstadoSistema(
            estado="saludable" if modelo else "modelo no cargado",
            modelo_cargado=bool(modelo),
            ultimo_procesamiento=ultimo_procesamiento,
            registros_procesados_hoy=registros_hoy,
            anomalias_detectadas_hoy=anomalias_hoy,
            espacio_disco_gb=round(espacio_gb, 2)
        )

    except Exception as e:
        logger.error(f"Error verificando salud: {e}")
        return EstadoSistema(
            estado="error",
            modelo_cargado=False,
            ultimo_procesamiento=None,
            registros_procesados_hoy=0,
            anomalias_detectadas_hoy=0,
            espacio_disco_gb=0
        )


@app.get("/logs/recientes", tags=["Monitoreo"])
async def obtener_logs_recientes(limite: int = Query(100, description="Número de logs a retornar")):
    """
    Obtiene los logs más recientes del sistema
    """
    try:
        logs = []
        # Leer todos los JSON de logs
        for archivo in os.listdir(LOG_PATH):
            if archivo.endswith(".json"):
                with open(os.path.join(LOG_PATH, archivo), "r") as f:
                    contenido = json.load(f)
                if isinstance(contenido, list):
                    logs.extend(contenido)
                else:
                    logs.append(contenido)

        # Ordenar y recortar
        logs_ordenados = sorted(
            logs,
            key=lambda x: x.get("timestamp", ""),
            reverse=True
        )[:limite]

        return {
            "total_logs": len(logs_ordenados),
            "logs": logs_ordenados
        }

    except Exception as e:
        logger.error(f"Error obteniendo logs: {e}")
        return {"total_logs": 0, "logs": []}


@app.delete("/limpiar/archivos-antiguos", tags=["Monitoreo"])
async def limpiar_archivos_antiguos(dias_retener: int = Query(7, description="Días a retener")):
    """
    Limpia archivos más antiguos que los días especificados
    """
    try:
        fecha_limite = datetime.now() - timedelta(days=dias_retener)
        archivos_eliminados = 0
        espacio_liberado = 0

        # OUTPUT_PATH
        for archivo in os.listdir(OUTPUT_PATH):
            ruta = os.path.join(OUTPUT_PATH, archivo)
            if os.path.isfile(ruta):
                mod_time = datetime.fromtimestamp(os.path.getmtime(ruta))
                if mod_time < fecha_limite:
                    tamaño = os.path.getsize(ruta)
                    os.remove(ruta)
                    archivos_eliminados += 1
                    espacio_liberado += tamaño

        # TEMP_PATH
        for archivo in os.listdir(TEMP_PATH):
            ruta = os.path.join(TEMP_PATH, archivo)
            if os.path.isfile(ruta):
                mod_time = datetime.fromtimestamp(os.path.getmtime(ruta))
                if mod_time < fecha_limite:
                    tamaño = os.path.getsize(ruta)
                    os.remove(ruta)
                    archivos_eliminados += 1
                    espacio_liberado += tamaño

        espacio_mb = espacio_liberado / (1024**2)
        return {
            "mensaje": "Limpieza completada",
            "archivos_eliminados": archivos_eliminados,
            "espacio_liberado_mb": round(espacio_mb, 2),
            "fecha_limite": fecha_limite.strftime("%Y-%m-%d")
        }

    except Exception as e:
        logger.error(f"Error limpiando archivos: {e}")
        raise HTTPException(status_code=500, detail=str(e))

## 10. ENDPOINTS DE INFORMACIÓN DE PAÍSES

In [17]:
@app.get("/paises/lista", tags=["Información"])
async def listar_paises():
    """
    Lista todos los países conocidos por el modelo
    """
    if not stats_dict:
        raise HTTPException(status_code=503, detail="Modelo no disponible")

    paises = []
    for pais, stats in stats_dict.items():
        paises.append({
            "codigo": pais,
            "categoria": stats.get("CATEGORIA", "Desconocido"),
            "registros_historicos": stats.get("REGISTROS", 0),
            "promedio_llamadas": round(stats.get("LLAMADAS_MEAN", 0), 2),
            "promedio_minutos": round(stats.get("MINUTOS_MEAN", 0), 2),
            "promedio_destinos": round(stats.get("DESTINOS_MEAN", 0), 2),
        })

    return {
        "total_paises": len(paises),
        "paises": sorted(
            paises,
            key=lambda x: x["registros_historicos"],
            reverse=True
        )
    }


@app.get("/paises/{codigo_pais}/estadisticas", tags=["Información"])
async def estadisticas_pais(codigo_pais: str):
    """
    Obtiene estadísticas detalladas de un país específico
    """
    if not stats_dict:
        raise HTTPException(status_code=503, detail="Modelo no disponible")

    if codigo_pais not in stats_dict:
        raise HTTPException(
            status_code=404,
            detail=f"País {codigo_pais} no encontrado"
        )

    stats = stats_dict[codigo_pais]

    return {
        "codigo_pais": codigo_pais,
        "categoria": stats.get("CATEGORIA"),
        "contexto_historico": contexto_historico.get(codigo_pais, "No disponible"),
        "estadisticas": {
            "registros": stats.get("REGISTROS", 0),
            "llamadas": {
                "promedio": round(stats.get("LLAMADAS_MEAN", 0), 2),
                "desviacion": round(stats.get("LLAMADAS_STD", 0), 2),
                "percentil_90": round(stats.get("LLAMADAS_P90", 0), 2),
                "percentil_95": round(stats.get("LLAMADAS_P95", 0), 2),
            },
            "minutos": {
                "promedio": round(stats.get("MINUTOS_MEAN", 0), 2),
                "desviacion": round(stats.get("MINUTOS_STD", 0), 2),
                "percentil_90": round(stats.get("MINUTOS_P90", 0), 2),
                "percentil_95": round(stats.get("MINUTOS_P95", 0), 2),
            },
            "destinos": {
                "promedio": round(stats.get("DESTINOS_MEAN", 0), 2),
                "desviacion": round(stats.get("DESTINOS_STD", 0), 2),
                "percentil_90": round(stats.get("DESTINOS_P90", 0), 2),
                "percentil_95": round(stats.get("DESTINOS_P95", 0), 2),
            },
        }
    }


# ##NUEVO ENDPOINT PARA CONSULTAR APRENDISAJE INCREMENTAL

In [18]:
@app.get("/aprendizaje/historial", tags=["Aprendizaje Incremental"])
async def historial_aprendizaje(
    limite: int = Query(30, description="Número de registros a mostrar")
):
    """
    Obtiene el historial de aprendizajes incrementales realizados
    """
    if not oracle_pool:
        raise HTTPException(status_code=503, detail="Oracle no disponible")
    
    try:
        with get_oracle_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT 
                    ID_APRENDIZAJE,
                    FECHA_EJECUCION,
                    FECHA_DATOS,
                    TOTAL_REGISTROS,
                    REGISTROS_PROCESADOS,
                    PAISES_NUEVOS,
                    PAISES_ACTUALIZADOS,
                    UMBRAL_ANTERIOR,
                    UMBRAL_NUEVO,
                    PAISES_CON_CAMBIO_CATEGORIA,
                    TIEMPO_PROCESAMIENTO_SEG,
                    ESTADO
                FROM APRENDIZAJE_INCREMENTAL
                ORDER BY FECHA_EJECUCION DESC
                FETCH FIRST :limite ROWS ONLY
            """, [limite])
            
            columnas = [col[0] for col in cursor.description]
            historiales = []
            
            for row in cursor:
                hist = dict(zip(columnas, row))
                # Convertir timestamps
                if hist['FECHA_EJECUCION']:
                    hist['FECHA_EJECUCION'] = hist['FECHA_EJECUCION'].isoformat()
                if hist['FECHA_DATOS']:
                    hist['FECHA_DATOS'] = hist['FECHA_DATOS'].isoformat()
                historiales.append(hist)
            
            return {
                "total": len(historiales),
                "historiales": historiales
            }
            
    except Exception as e:
        logger.error(f"Error obteniendo historial de aprendizaje: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/aprendizaje/{id_aprendizaje}/detalle", tags=["Aprendizaje Incremental"])
async def detalle_aprendizaje(id_aprendizaje: int):
    """
    Obtiene el detalle completo de un aprendizaje incremental específico
    """
    if not oracle_pool:
        raise HTTPException(status_code=503, detail="Oracle no disponible")
    
    try:
        with get_oracle_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT * FROM APRENDIZAJE_INCREMENTAL
                WHERE ID_APRENDIZAJE = :id
            """, [id_aprendizaje])
            
            row = cursor.fetchone()
            if not row:
                raise HTTPException(status_code=404, detail="Aprendizaje no encontrado")
            
            columnas = [col[0] for col in cursor.description]
            detalle = dict(zip(columnas, row))
            
            # Convertir timestamps y CLOB
            if detalle['FECHA_EJECUCION']:
                detalle['FECHA_EJECUCION'] = detalle['FECHA_EJECUCION'].isoformat()
            if detalle['FECHA_DATOS']:
                detalle['FECHA_DATOS'] = detalle['FECHA_DATOS'].isoformat()
            
            # Parsear JSON de cambios de categoría si existe
            if detalle['DETALLE_CAMBIOS_CATEGORIA']:
                try:
                    detalle['DETALLE_CAMBIOS_CATEGORIA'] = json.loads(
                        detalle['DETALLE_CAMBIOS_CATEGORIA'].read()
                    )
                except:
                    detalle['DETALLE_CAMBIOS_CATEGORIA'] = {}
            
            return detalle
            
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"Error obteniendo detalle de aprendizaje: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/aprendizaje/estadisticas-evolucion", tags=["Aprendizaje Incremental"])
async def estadisticas_evolucion():
    """
    Muestra la evolución del modelo a través de los aprendizajes incrementales
    """
    if not oracle_pool:
        raise HTTPException(status_code=503, detail="Oracle no disponible")
    
    try:
        with get_oracle_connection() as conn:
            cursor = conn.cursor()
            
            # Evolución del umbral
            cursor.execute("""
                SELECT 
                    TO_CHAR(FECHA_EJECUCION, 'YYYY-MM-DD') as fecha,
                    UMBRAL_ANTERIOR,
                    UMBRAL_NUEVO
                FROM APRENDIZAJE_INCREMENTAL
                WHERE ESTADO = 'COMPLETADO'
                ORDER BY FECHA_EJECUCION
            """)
            
            evolucion_umbral = [
                {
                    "fecha": row[0],
                    "umbral_anterior": float(row[1]) if row[1] else None,
                    "umbral_nuevo": float(row[2]) if row[2] else None
                }
                for row in cursor.fetchall()
            ]
            
            # Evolución de países conocidos
            cursor.execute("""
                SELECT 
                    TO_CHAR(FECHA_EJECUCION, 'YYYY-MM-DD') as fecha,
                    PAISES_CONOCIDOS_ANTERIOR,
                    PAISES_CONOCIDOS_NUEVO,
                    PAISES_NUEVOS
                FROM APRENDIZAJE_INCREMENTAL
                WHERE ESTADO = 'COMPLETADO'
                ORDER BY FECHA_EJECUCION
            """)
            
            evolucion_paises = [
                {
                    "fecha": row[0],
                    "paises_antes": row[1],
                    "paises_despues": row[2],
                    "paises_nuevos": row[3]
                }
                for row in cursor.fetchall()
            ]
            
            # Estadísticas agregadas
            cursor.execute("""
                SELECT 
                    COUNT(*) as total_ejecuciones,
                    SUM(REGISTROS_PROCESADOS) as total_registros,
                    SUM(PAISES_NUEVOS) as total_paises_nuevos,
                    AVG(TIEMPO_PROCESAMIENTO_SEG) as tiempo_promedio,
                    MAX(FECHA_EJECUCION) as ultima_ejecucion
                FROM APRENDIZAJE_INCREMENTAL
                WHERE ESTADO = 'COMPLETADO'
            """)
            
            stats = cursor.fetchone()
            estadisticas_generales = {
                "total_ejecuciones": stats[0] or 0,
                "total_registros_procesados": stats[1] or 0,
                "total_paises_descubiertos": stats[2] or 0,
                "tiempo_promedio_segundos": float(stats[3]) if stats[3] else 0,
                "ultima_actualizacion": stats[4].isoformat() if stats[4] else None
            }
            
            return {
                "estadisticas_generales": estadisticas_generales,
                "evolucion_umbral": evolucion_umbral,
                "evolucion_paises": evolucion_paises
            }
            
    except Exception as e:
        logger.error(f"Error obteniendo estadísticas de evolución: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/aprendizaje/programar-diario", tags=["Aprendizaje Incremental"])
async def programar_aprendizaje_diario(
    hora_ejecucion: str = Query("02:00", description="Hora de ejecución diaria (HH:MM)"),
    directorio_datos: str = Query(..., description="Directorio donde se encuentran los datos diarios")
):
    """
    Programa el aprendizaje incremental diario (requiere implementación de scheduler)
    """
    # Esta es una función placeholder que muestra cómo se podría implementar
    return {
        "mensaje": "Funcionalidad de programación diaria",
        "nota": "Requiere implementar un scheduler (como APScheduler) para ejecución automática",
        "configuracion_sugerida": {
            "hora_ejecucion": hora_ejecucion,
            "directorio_datos": directorio_datos,
            "comando_cron": f"0 {hora_ejecucion.split(':')[1]} {hora_ejecucion.split(':')[0]} * * python ejecutar_aprendizaje_diario.py"
        }
    }

## 11. CONFIGURACIÓN PARA EJECUTAR LA API


In [19]:
def iniciar_api_notebook():
    """Inicia la API en modo notebook"""
    import nest_asyncio
    nest_asyncio.apply()
    import uvicorn

    print("\n🚀 INICIANDO API DE DETECCIÓN DE FRAUDE")
    print("="*60)
    print("📍 URL Base: http://localhost:8000")
    print("📚 Documentación: http://localhost:8000/docs")
    print("📊 Documentación alternativa: http://localhost:8000/redoc")
    print("="*60)
    print("\nEndpoints principales:")
    print("  POST   /scoring/individual        - Scoring de un registro")
    print("  POST   /scoring/batch            - Scoring de archivo CSV")
    print("  POST   /modelo/aprendizaje-incremental - Actualizar modelo")
    print("  GET    /modelo/estadisticas      - Ver estadísticas del modelo")
    print("  GET    /analisis/resumen-diario  - Resumen por día")
    print("  GET    /salud                    - Estado del sistema")
    print("="*60)
    print("\n⏹️  Para detener: Interrumpir el kernel del notebook\n")

    # Ejecutar servidor
    uvicorn.run(app, host="0.0.0.0", port=8000, reload=False)

In [ ]:
iniciar_api_notebook()


INFO:     Started server process [24908]
INFO:     Waiting for application startup.
2025-06-18 20:33:25,369 - __main__ - INFO - Iniciando API...



🚀 INICIANDO API DE DETECCIÓN DE FRAUDE
📍 URL Base: http://localhost:8000
📚 Documentación: http://localhost:8000/docs
📊 Documentación alternativa: http://localhost:8000/redoc

Endpoints principales:
  POST   /scoring/individual        - Scoring de un registro
  POST   /scoring/batch            - Scoring de archivo CSV
  POST   /modelo/aprendizaje-incremental - Actualizar modelo
  GET    /modelo/estadisticas      - Ver estadísticas del modelo
  GET    /analisis/resumen-diario  - Resumen por día
  GET    /salud                    - Estado del sistema

⏹️  Para detener: Interrumpir el kernel del notebook



2025-06-18 20:33:26,576 - __main__ - INFO - Modelo cargado exitosamente
2025-06-18 20:33:26,578 - __main__ - INFO - Pool de conexiones Oracle inicializado
2025-06-18 20:33:26,654 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,658 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,662 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,667 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,671 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,673 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,676 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,679 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,683 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,686 - __main__ - INFO - Tabla ya existe, continuando...
2025-06-18 20:33:26,687 - __main__ - INFO - Todas las tablas creadas/verificadas exit

INFO:     127.0.0.1:58801 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:58801 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:58802 - "POST /modelo/aprendizaje-incremental HTTP/1.1" 200 OK


2025-06-18 20:33:51,730 - __main__ - INFO - Iniciando aprendizaje incremental con 102684 registros


...

INFO:     127.0.0.1:59564 - "POST /modelo/recargar HTTP/1.1" 200 OK
INFO:     127.0.0.1:59564 - "GET /modelo/estadisticas HTTP/1.1" 200 OK
INFO:     127.0.0.1:59565 - "GET /aprendizaje/historial?limite=30 HTTP/1.1" 200 OK


2025-06-18 20:42:17,761 - __main__ - INFO - Modelo cargado exitosamente


INFO:     127.0.0.1:59669 - "POST /modelo/recargar HTTP/1.1" 200 OK
INFO:     127.0.0.1:59669 - "GET /modelo/estadisticas HTTP/1.1" 200 OK


2025-06-18 20:42:44,161 - __main__ - INFO - Modelo cargado exitosamente


INFO:     127.0.0.1:59670 - "POST /modelo/recargar HTTP/1.1" 200 OK
INFO:     127.0.0.1:59670 - "GET /modelo/estadisticas HTTP/1.1" 200 OK
INFO:     127.0.0.1:59716 - "GET /lotes/historial?limite=50 HTTP/1.1" 200 OK
